In [1]:
import codecs, json
import numpy as np
import pandas as pd
import glob
import pickle
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [2]:
data_path = 'data/'
files = glob.glob(data_path + '/*')
train = pickle.load(open(files[1], 'rb'))
test = pickle.load(open(files[0], 'rb'))
d_train = pd.DataFrame({'text': train[0], 'kategori': train[1]})
d_test = pd.DataFrame({'text': test[0], 'kategori': test[1]})
githubdata = pd.concat([d_train, d_test], ignore_index=True, sort =False)
# githubdata = githubdata.loc[0:10]
# githubdata = githubdata.loc[githubdata['kategori'] == 'bisnis'].reset_index(drop=True)
print(len(githubdata))

8754


In [3]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from preprocess import prep
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# import StopWordRemoverFactory class
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
stoplist = pd.read_csv('stoplist.csv')

# Ambil Stopword bawaan
stop_factory = StopWordRemoverFactory().get_stop_words()

# Custom stopword
data = stoplist['list'].tolist()

dictionary = ArrayDictionary(data)
stopword = StopWordRemover(dictionary)

# CREATING VOCAB & WORD COUNTS

In [4]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
#get the text column 
docs=githubdata['text'].tolist()
cv=CountVectorizer(max_df=0.85,stop_words=data,max_features=10000)
word_count_vector=cv.fit_transform(docs)
# list(cv.vocabulary_.keys())[:10]

c:\python37\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


# TfidfTransformer to Compute Inverse Document Frequency (IDF)

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
# print(tfidf_transformer)

TfidfTransformer()

# Computing TF-IDF and Extracting keywords

In [7]:
# # get test docs into a list
# docs_test=githubdata['stemmed'].tolist()

In [8]:
# # you only needs to do this once, this is a mapping of index to 
# feature_names=cv.get_feature_names()

# # get the document that we want to extract keywords from
# doc=docs_test[1]

# #generate tf-idf for the given document
# tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

# #sort the tf-idf vectors by descending order of scores
# sorted_items=sort_coo(tf_idf_vector.tocoo())

# #extract only the top n; n here is 10
# keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# # now print the results
# print("\n=====Doc=====")
# print(doc)
# print("\n===Keywords===")
# keywords_list = []
# print(keywords)
# for k in keywords:
#     keywords_list.append(k)
# print(keywords_list)


# PREPROCESSING

In [9]:
for i in tqdm(range(len(githubdata))):
    githubdata.loc[i, 'stemmed'] = stemmer.stem(stopword.remove(prep(githubdata.loc[i, 'text'])))
    docs_test=githubdata['stemmed'].tolist()
    # you only needs to do this once, this is a mapping of index to 
    feature_names=cv.get_feature_names()

    # get the document that we want to extract keywords from
    doc=docs_test[i]

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
#     print(tf_idf_vector)

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    keywords_list = []
    for k in keywords:
        keywords_list.append(k)
#     print(keywords_list)
    githubdata.loc[i, 'keywords'] = ",".join(keywords_list)
# githubdata = githubdata.drop(columns=['text']) # Get rid of the unstemmed column.
# githubdata.to_json('bigdata.json', orient='records', lines=True)

100%|████████████████████████████████████████████████████████████████████████████| 8754/8754 [1:16:22<00:00,  1.91it/s]


# FIT & TRANSFORM LABEL

In [14]:
githubdata.head()
githubdata.to_csv("hasil.csv")

In [15]:
from sklearn.model_selection import train_test_split
X = cv.transform(githubdata['stemmed'].tolist())
y=githubdata['kategori']
x_train,x_test,y_train,y_test= train_test_split(X,y,
test_size=.075, random_state= 42) 

# TRAINING

In [16]:
import joblib
from sklearn.naive_bayes import MultinomialNB

# # Mengaktifkan/memanggil/membuat fungsi klasifikasi Naive Bayes dan OnVsRest
classifier = MultinomialNB()
# # Memasukkan data training
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
#hitung akurasi data test
score = classifier.score(x_test, y_test)
# all_labels = mlb.inverse_transform(pred)
modelfile = 'modelNB-real.sav'
labelfile = 'vectorizer-real.sav'
joblib.dump(classifier, modelfile)
joblib.dump(cv, labelfile)
for item, labels in zip(cv.inverse_transform(x_test), y_test):
    print('{0} => {1}'.format(" ".join(item),labels))


aditya alami arus asing aspek bagus banding bank bayar bi bijak bukti capai capital dana deras dwi ekonomi februari febuari global gubernur harga indonesia inflow investor jakarta jasa jumat khusus kompleks kuartal langkah lepas masjid masuk modal negara neraca otoritas pasar percaya perintah periode perry portofolio prospek pungkas putra reporter rp saham sbn sertifikat sesuai sumber surat surplus tempuh temu tingkat to total triliun uang warjiyo year => bisnis
abang acara amar berita deradikalisasi februari hercules indonesia izin jakarta jaksa jatuh jenderal kanal keliru lahan lantas mantan marsekal marshal masyarakat menhan menteri mindset negara news nilai panglima penjara polri populer program provokasi purn putus rencana rosario ryamizard salah selasa sikap sisi susun tahan takut tanah terorisme tjahjanto tni top ubah utuh vonis => news
agung alami aluminium angkut atur bara batu batubara bentuk bijak biro dasar daya dmt energi esdm export februari free harga hitung imbuh impor 